In [ ]:
import pandas as pd
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from nltk import ngrams
from tqdm import tqdm
import numpy as np
import itertools
import random

pd.options.display.float_format = '{:20,.2f}'.format

In [ ]:
df = pd.read_csv('good.csv', error_bad_lines=False)
df['len_text'] = df['text'].str.split().str.len()
df.head()

## Create data

In [ ]:
texts = list(df['text'].values[0:2000])
texts = ' '.join(texts).split('.')
texts = [sentence for sentence in texts if len(sentence.split())>10]
total_texts = list(df['text'].values)
def duplicate_function(text):
    number = random.uniform(0, 1)
    if number > 0.75:
        text = text.split('. ')
        for i in range(int(len(text) * 0.1)):
            text.insert(random.randint(0, len(text)), random.choice(texts))
        result = pd.Series(['. '.join(text), 1])
        return result
    else:
        result = pd.Series([random.choice(total_texts), 0])
        return result

In [ ]:
df[['new_text', 'deduplicate']] = df['text'].apply(duplicate_function)
df['len_text'] = df['text'].str.split().str.len()
df['len_new_text'] = df['new_text'].str.split().str.len()
df['diff_similarity'] = np.abs(1 - (df['len_new_text'] / df['len_text'])) * 100
df = df[df['text'] != df['new_text']]
df_1 = df[['text', 'deduplicate', 'diff_similarity']]
df_1[['deduplicate', 'diff_similarity']] = 0
df_2 = df[df['deduplicate'] == 1][['new_text', 'deduplicate', 'diff_similarity']]
df_2.rename(columns={'new_text':'text'}, inplace=True)
df = pd.concat([df_1, df_2])
df = df.reset_index().rename(columns={'index':'original'})
df.head()

In [ ]:
df.to_csv('Dataset_duplicate_test.csv', index=None)